In [1]:
from restrictions import *

In [2]:
# Base categories
FoodCategory("MEAT")
FoodCategory("DAIRY")
FoodCategory("EGGS")
FoodCategory("SHELLFISH")
FoodCategory("FISH")
FoodCategory("NUTS")
FoodCategory("GLUTEN")

# Subcategories
FoodCategory("BEEF", {"MEAT"})
FoodCategory("CHICKEN", {"MEAT"})
FoodCategory("CHEESE", {"DAIRY"})

# Composite/umbrella category
FoodCategory("ANIMAL_PRODUCTS", {"MEAT", "DAIRY", "EGGS", "FISH"})


FoodCategory(ANIMAL_PRODUCTS)

In [ ]:
beef = FoodCategory.get("BEEF")
print(beef.is_a("MEAT"))            # True
print(beef.is_a("ANIMAL_PRODUCTS")) # True
print(beef.is_a("NUTS"))            # False

animal_products = FoodCategory.get("ANIMAL_PRODUCTS")
print(animal_products.ancestors())            # set()


True
False
False
{'MEAT'}
